## Adjusting accessibility statistics

In [1]:
import psycopg2
import configparser
import pandas as pd
import numpy as np

config = configparser.ConfigParser()
config.read("../../config.ini")    
db_params = dict(config['DB'])
from sqlalchemy import create_engine

def execute_sql(SQL):
        with psycopg2.connect(**db_params) as conn:
            with conn.cursor() as cur:
                cur.execute(SQL)        


def get_alchemy_engine():
    conn_string = 'postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**db_params)
    return create_engine(conn_string, echo=False)

### Creating the function that calculates the adjustment required

In [2]:
SQL = """
DROP FUNCTION api_add_remove_catchments;

CREATE OR REPLACE FUNCTION api_add_remove_catchments(
    in_remove_hex_ids character[],
    in_add_hex_ids character[],		
	in_timeofday varchar,
	in_categorytype text
)
    RETURNS TABLE
            (
                h3id         character,                                
                adjustment        float
            )
    LANGUAGE plpgsql
AS
$addremovecatchments$
BEGIN
    RETURN QUERY
		--- Find all distinct catchment IDs where POIs are to be removed
        WITH remove_catchment_ids AS (
		        SELECT DISTINCT catchmentid,
				-1 as sign
		        FROM catchments 		        
		        JOIN unnest(in_remove_hex_ids) m(id) ON catchments.originh3id = m.id
		    ),
		--- Find all distinct catchment IDs where POIs are to be added
		add_catchment_ids AS (
			SELECT catchmentid,
				COUNT(*) as sign -- to allow for adding multiple POIs in the same hexagon ID
		        FROM catchments 		        
		        JOIN unnest(in_add_hex_ids) m(id) ON catchments.originh3id = m.id
                GROUP BY catchmentid
			),
			
		-- Merge them together
		all_catchment_ids AS (
			SELECT catchmentid, SUM(sign) as sign FROM 
			(SELECT * FROM add_catchment_ids UNION SELECT * FROM remove_catchment_ids) as t
			GROUP BY catchmentid
		)
            
		--- Obtain step 1 values to be adjusted		
		SELECT 
			step1.h3id, 			
			sum(ratio * sign) as ratio -- sign is + if adding, - if removing
			FROM step1_stats as step1
			JOIN all_catchment_ids ON step1.catchmentid = all_catchment_ids.catchmentid			
			WHERE 
				categorytype = in_categorytype
				AND
				timeofday = in_timeofday				
			GROUP BY step1.h3id;
		

END;
$addremovecatchments$;

"""

execute_sql(SQL)

### Check #1 - adding and removing the same catchment area

Hex ID 8944c106d37ffff contains multiple restaurants - despite that, the adjustment should be zero!

In [4]:
id = '8944c106d37ffff'

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute("""
        SELECT *
            FROM api_add_remove_catchments(
	        array [%s], 
	        array [%s],	        
	        %s,
	        %s)
        """, (id, id, 'morning', 'Race'))
        res = cur.fetchall()

df = pd.DataFrame(res, columns=['hexid', 'adjustment'])
assert (np.all(np.isclose(df.adjustment.values, 0))), "Not all values are zero - check!"
print("Test passed")

Test passed


### Check #2 - adding a catchment area should result only in positive adjustments

In [6]:
id = ['8944c1a8133ffff']

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute("""
        SELECT *
            FROM api_add_remove_catchments(	        
	        array [%s],
            %s, 	        
	        %s,
	        %s)
        """, ("", [id], 'morning', 'Race'))
        res = cur.fetchall()

df = pd.DataFrame(res, columns=['hexid', 'adjustment'])
assert (np.all(np.isclose(df.adjustment.values > 0, True))), "Not all values are larger than zero - check!"
assert (np.all(np.isclose(df.adjustment.values - df.adjustment.values[0], 0))), "Not all values are identical - check!"
print("Tests passed")
df

Tests passed


,hexid,adjustment
0,8944c1a8e93ffff,0.07674
1,8944c1a83cfffff,0.07674
2,8944c1a164fffff,0.07674
3,8944c1aaa4fffff,0.07674
4,8944c1a9cd3ffff,0.07674
...,...,...
519,8944c1a81a3ffff,0.07674
520,8944c1aaa2bffff,0.07674
521,8944c1a81cfffff,0.07674
522,8944c1a8103ffff,0.07674


### Check #3 - removing catchment area should result in identical negative adjustment everywhere

In [8]:
id = '8944c106d37ffff'

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute("""
        SELECT *
            FROM api_add_remove_catchments(	        	        
            array [%s], 
            array [%s],	        
	        %s,
	        %s)
        """, (id, "", 'morning', 'Race'))
        res = cur.fetchall()

df = pd.DataFrame(res, columns=['hexid', 'adjustment'])
assert (np.all(np.isclose(df.adjustment.values < 0, True))), "Not all values are smaller than zero - check!"
assert (np.all(np.isclose(df.adjustment.values - df.adjustment.values[0], 0))), "Not all values are identical - check!"
print("Tests passed")

Tests passed
